In [ ]:
pip install mnemonic hdwallet

In [ ]:
import bitcoin
import random
from mnemonic import Mnemonic
from hdwallet import BIP44HDWallet
from hdwallet.derivations import P2PKH

# Charger les adresses
with open('btc.txt', 'r') as file:
    addresses_set = set(line.strip() for line in file)

# Charger les mots bip39
with open('words.txt', 'r') as file:
    bip39_words = [line.strip() for line in file if line.strip()]

def is_address_in_set(address):
    return address in addresses_set

def generate_keypair_from_mnemonic(mnemonic):
    # Générer un HD Wallet à partir de la mnemonic
    bip44_wallet = BIP44HDWallet.derive_from_mnemonic(mnemonic, language="english")
    bip44_wallet.from_path(P2PKH.BIP44_CHANGE_CHAIN.format(c=0))
    private_key = bip44_wallet.private_key()
    address = bip44_wallet.address()
    return private_key, address

def generate_bitcoin_keypair_with_prefix(prefixes):
    attempt_count = 0
    file_count = 1
    entries_in_current_file = 0

    while True:
        # Sélection des mots bip39
        words_count = random.choice([3, 6, 12, 24])
        selected_words = random.sample(bip39_words, words_count)
        mnemonic = ' '.join(selected_words)

        # Générer clé privée et adresse à partir de mnemonic
        private_key, address = generate_keypair_from_mnemonic(mnemonic)

        # ... [Code précédent]

def generate_bitcoin_keypair_with_prefix(prefixes):
    attempt_count = 0
    file_count = 1
    entries_in_current_file = 0

    while True:
        # Sélection des mots bip39
        words_count = random.choice([3, 6, 12, 24])
        selected_words = random.sample(bip39_words, words_count)
        mnemonic = ' '.join(selected_words)

        # Générer clé privée et adresse à partir de mnemonic
        private_key, address = generate_keypair_from_mnemonic(mnemonic)

        attempt_count += 1
        entries_in_current_file += 1
        print(f"Adresse essayée : {address}")

        # Écrire dans random_N.txt tous les 50 000 essais
        if entries_in_current_file > 50000:
            entries_in_current_file = 1
            file_count += 1

        wif_private_key = bitcoin.encode_privkey(bitcoin.decode_privkey(private_key, 'hex'), 'wif')
        with open(f'random_{file_count}.txt', 'a') as rand_file:
            rand_file.write(f"Essai n°{attempt_count}\n")
            rand_file.write(f"Clé privée hex : {private_key}\n")
            rand_file.write(f"Clé privée WIF : {wif_private_key}\n")
            rand_file.write(f"Adresse : {address}\n\n")

        if is_address_in_set(address):
            # Si une correspondance est trouvée, on écrit les détails dans resultat.txt
            with open('resultat.txt', 'a') as file:
                file.write(f"Clé privée hex : {private_key}\n")
                file.write(f"Clé privée WIF : {wif_private_key}\n")
                file.write(f"Adresse : {address}\n\n")
            return {
                "private_key_hex": private_key,
                "wif_private_key": wif_private_key,
                "address": address
            }

prefixes_to_check = ["1", "3", "bc1"]
kp = generate_bitcoin_keypair_with_prefix(prefixes_to_check)

print("Clé privée hex :", kp["private_key_hex"])
print("Clé privée WIF :", kp["wif_private_key"])
print(f"Adresse :", kp["address"])
